<a href="https://colab.research.google.com/github/Sundaynot/air-quality/blob/main/src/db_initial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Clonazione della repo se non esiste**

In [4]:
import os
import pandas as pd

# Nome della cartella del repo
repo_name = "air-quality"
repo_url = "https://github.com/sundaynot/air-quality.git"
notebook_path = "src"
csv_path = "../data/Air_Quality.csv"

# 🔁 Clona il repo solo se non esiste già
if not os.path.exists(repo_name):
    print("📥 Clonazione del repository...")
    !git clone {repo_url}
else:
    print("✅ Repository già presente.")

📥 Clonazione del repository...
Cloning into 'air-quality'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 43 (delta 14), reused 10 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (43/43), 978.45 KiB | 11.51 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [5]:
%cd air-quality/src
print(os.getcwd())
#entro nel percorso corretto

/content/air-quality/src
/content/air-quality/src


Visualizzo le prime 5 righe del Database


In [6]:
df = pd.read_csv('../data/Air_Quality.csv')
df.head()

,Date,City,CO,NO2,SO2,O3,PM2.5,PM10,AQI
0,2023-01-01 00:00:00+00:00,Brasilia,292.0,30.15,1.10,23.0,11.1,15.8,21.158330
1,2023-01-01 01:00:00+00:00,Brasilia,291.0,28.95,1.15,24.0,12.4,17.7,20.825000
2,2023-01-01 02:00:00+00:00,Brasilia,289.0,27.25,1.25,25.0,13.0,18.5,20.474998
3,2023-01-01 03:00:00+00:00,Brasilia,275.0,23.70,1.30,28.0,9.2,13.1,20.074999
4,2023-01-01 04:00:00+00:00,Brasilia,246.0,19.35,1.25,32.0,6.8,9.7,19.324999


In [7]:
df.describe()  #alcune statistiche sul database

,CO,NO2,SO2,O3,PM2.5,PM10,AQI
count,52560.000000,52560.000000,52560.000000,52560.000000,52560.000000,52560.000000,52560.000000
mean,273.372565,28.930514,12.051645,55.304433,18.621792,35.505850,41.941352
std,171.578815,25.473936,14.684127,36.939938,15.897499,41.866614,24.989747
min,48.000000,0.000000,0.000000,0.000000,0.000000,0.100000,8.000000
25%,160.000000,11.100000,2.500000,29.000000,7.100000,11.000000,22.638333
50%,224.000000,22.100000,5.800000,51.000000,13.400000,20.800000,32.400000
75%,335.000000,38.700000,17.300000,74.000000,24.900000,41.700000,61.013332
max,2145.000000,251.900000,200.800000,425.000000,143.400000,550.000000,188.319980


In [8]:
df.shape  #restituisce il numero di righe e colonne del database


(52560, 9)